# Thêm lối tắt folder ở dưới vào drive trước khi chạy, nhớ chỉnh VID_TO_HANDLE ở phần 1 theo video muốn chạy transnet, ví dụ chạy video 1, 2, 3 thì chỉnh VID_TO_HANDLE = (1, 2, 3)
link folder: https://drive.google.com/drive/folders/1l5m5ghsoH7Ajsvd02P7nAjY7rmNXClLl?usp=drive_link

# 0. Set up environment

In [ ]:
!apt-get install ffmpeg
!pip install ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 362 (delta 4), reused 4 (delta 4), pack-reused 355
Receiving objects: 100% (362/362), 98.15 KiB | 19.63 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Filtering content: 100% (3/3), 34.77 MiB | 10.93 MiB/s, done.


In [ ]:
import os
import numpy as np
import tensorflow as tf
import shutil
import zipfile
import cv2
import pandas as pd
from tqdm import tqdm
import multiprocessing
from multiprocessing import Manager
from multiprocessing.pool import Pool
import glob
from TransNetV2.inference.transnetv2 import TransNetV2
import math
from collections import OrderedDict

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
DATA_PATH = 'data'
SAVE_PATH = 'result'
DRIVE_PATH = 'drive/MyDrive/AI_Challenge'
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)
if not os.path.exists(SAVE_PATH):
    os.mkdir(SAVE_PATH)

# 1. Load Data

In [ ]:
VID_TO_HANDLE = (1, 2, 3, 4) # Indexes of videos to infer

In [ ]:
# Copy videos from drive and extract
for i in VID_TO_HANDLE:
    vid_path = f'Videos_L0{i}.zip' if i < 10 else f'Videos_L{i}.zip'
    shutil.copy(os.path.join(DRIVE_PATH, 'Videos', vid_path), '.')
    if i <= 10:
        save_path = DATA_PATH
    else:
        save_path = os.path.join(DATA_PATH, vid_path.split('.')[0])
        if not os.path.exists(save_path):
            os.mkdir(save_path)
    with zipfile.ZipFile(vid_path, 'r') as zip_ref:
        zip_ref.extractall(save_path)
    os.remove(vid_path)

# 2. Inference

In [ ]:
# function to save frames at specific idxs in a given video
def save_frame_at_idx(vid_path, save_path, idx):
    vid = cv2.VideoCapture(vid_path)
    for i in idx:
        vid.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = vid.read()
        cv2.imwrite(os.path.join(save_path, f'{i}.png'), frame)
    vid.release()

In [ ]:
# function to zip, move result to drive and clean memory
def zip2drive(vid_pack):
    shutil.rmtree(os.path.join(DATA_PATH, vid_pack))
    path = os.path.join(SAVE_PATH, vid_pack)
    shutil.make_archive(path, 'zip', path)

    save_dir = os.path.join(DRIVE_PATH, 'TransNetV2_result')
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    shutil.rmtree(path)
    shutil.move(path + '.zip', save_dir)

In [ ]:
model = TransNetV2()
result = None
move2drive = []
num_processes = multiprocessing.cpu_count() * 3
with Pool(num_processes) as pool:
    for video_pack in os.listdir(DATA_PATH):
        video_paths = glob.glob(os.path.join(DATA_PATH, video_pack, 'video', '*.mp4'))
        for video in video_paths:
            video_frames, single_frame_predictions, all_frame_predictions = \
                model.predict_video(video)

            save_path = SAVE_PATH
            for dir in video.split('/')[1:]:
                if dir == "video":
                    continue
                if dir.endswith('.mp4'):
                    save_path = os.path.join(save_path, dir[:-4])
                else:
                    save_path = os.path.join(save_path, dir)
                if not os.path.exists(save_path):
                    os.mkdir(save_path)

            # predictions = np.stack([single_frame_predictions, all_frame_predictions], 1)
            # np.savetxt(os.path.join(save_path, "predictions.txt"), predictions, fmt="%.6f")

            scenes = model.predictions_to_scenes(single_frame_predictions, threshold=0.3)
            np.savetxt(os.path.join(save_path, "scenes.txt"), scenes, fmt="%d")


            # pil_image = model.visualize_predictions(
            #     video_frames, predictions=(single_frame_predictions, all_frame_predictions))
            # pil_image.save(os.path.join(save_path, "vis.png"))

            frame_idx = []  # save all frames need to retrieve
            for scene in scenes:
                frame_idx.extend([scene[0], (scene[0] + scene[1]) // 2, scene[1]])
            frame_idx = list(OrderedDict.fromkeys(frame_idx)) # remove duplicated frames

            # split the frames indexes into multiple sublist for multiprocessing
            step = math.ceil(len(frame_idx) / num_processes)
            split_frame_idx = []
            while True:
              if len(frame_idx) <= step:
                  split_frame_idx.append(frame_idx)
                  break
              else:
                  split_frame_idx.append(frame_idx[:step])
                  frame_idx = frame_idx[step:]

            frames_save_dir = os.path.join(save_path, 'frames')
            if not os.path.exists(frames_save_dir):
                os.mkdir(frames_save_dir)

            if result is not None:
                result.wait() # wait for the previous processes to finish

            result = pool.starmap_async(
                save_frame_at_idx,
                [(video, os.path.join(save_path, 'frames'), idx) for idx in split_frame_idx],
            ) # save frames parallel

        if result is not None:
            result.wait()
            result = None
        p = multiprocessing.Process(target=zip2drive, args=(video_pack, )) # move result to drive silently
        p.start()
        move2drive.append(p)

    for p in move2drive:
        p.join() # wait for all processes to exit


[TransNetV2] Using weights from /content/TransNetV2/inference/transnetv2-weights/.
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V001.mp4
[TransNetV2] Processing video frames 30916/30916
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V028.mp4
[TransNetV2] Processing video frames 29047/29047
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V009.mp4
[TransNetV2] Processing video frames 27400/27400
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V004.mp4
[TransNetV2] Processing video frames 37065/37065
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V021.mp4
[TransNetV2] Processing video frames 27810/27810
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V006.mp4
[TransNetV2] Processing video frames 31028/31028
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V002.mp4
[TransNetV2] Processing video frames 31354/31354
[TransNetV2] Extracting frames from data/Videos_L05/video/L05_V005.mp4
[Trans